In [16]:
import practicalSPARQL
import pandas as pd
import ast
import matplotlib.pyplot as plt
import numpy as np
import os
import requests

In [22]:
# this notebook gets all elements with a SAC_SIL and CK assigned, downloads the images to the folder elements_images_dataset and saves the data as a csv as well

In [43]:
root_q = 'queries'
root_d = 'results'

# read login data
login = pd.read_json('config_sparql.json')
ENDPOINT = login['endpoint'][0]
USERNAME = login['username'][0]
PASSWORD = login['password'][0]

# create sparql object
sparql = practicalSPARQL.practicalWrapper(ENDPOINT)
sparql.setCredentials(USERNAME, PASSWORD)

print("--- Querying ENDPOINT: {} ---".format(ENDPOINT))

--- Querying ENDPOINT: http://devmeta.sphaera.mpiwg-berlin.mpg.de/sparql ---


In [49]:
q = practicalSPARQL.stringify_SPARQL('dataset_publication_query_270125.sparql')    # select data from the ttl file as a dataframe
elements = sparql.select_as_dataframe(q)

In [50]:
elements

,imageURL,page_link,bb,cluster_name,cks,doiv,short_title,bid,pagenum,authors,publishers,printers,pid,pdoiv,place,year
0,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=354,261,360,363",SAC_SIL_00197,CK_Equinoctial,21.11103/sphaera.101116,Iohannis de Sacro Busto libellus de sphaera,1898,43,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,932,21.11103/sphaera.101420,Wittenberg,1578
1,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=351,433,548,279",SAC_SIL_01942,"CK_Refraction, CK_Apparent Size of Stars, CK_S...",21.11103/sphaera.101116,Iohannis de Sacro Busto libellus de sphaera,1898,33,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,932,21.11103/sphaera.101420,Wittenberg,1578
2,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=309,692,597,128",SAC_SIL_01616,"CK_Apparent Size of Stars, CK_Sphericity of th...",21.11103/sphaera.101116,Iohannis de Sacro Busto libellus de sphaera,1898,33,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,932,21.11103/sphaera.101420,Wittenberg,1578
3,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=473,212,336,243",SAC_SIL_01871,"CK_Apparent Size of Stars, CK_Sphericity of th...",21.11103/sphaera.101116,Iohannis de Sacro Busto libellus de sphaera,1898,33,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,932,21.11103/sphaera.101420,Wittenberg,1578
4,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=601,214,340,329",SAC_SIL_01952,CK_Relations Between the Spheres of the Sun an...,21.11103/sphaera.101116,Iohannis de Sacro Busto libellus de sphaera,1898,29,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,932,21.11103/sphaera.101420,Wittenberg,1578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20949,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=616,1352,300,415",SAC_SIL_02926,"CK_Symbols of the Moon, CK_Symbols of Planets",21.11103/sphaera.101089,La sfera di M. Giovanni di Sacrobosco,2215,63,"Sacrobosco, Johannes de, Danti, Piervincenzo","Salviani, Baldo","Rastelli, Giovanni Bernardino",390,21.11103/sphaera.100250,Perugia,1574
20950,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=1065,613,594,594",SAC_SIL_01943,"CK_Circles of Equant Deferent Epicycle, CK_Lun...",21.11103/sphaera.101089,La sfera di M. Giovanni di Sacrobosco,2215,62,"Sacrobosco, Johannes de, Danti, Piervincenzo","Salviani, Baldo","Rastelli, Giovanni Bernardino",390,21.11103/sphaera.100250,Perugia,1574
20951,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=1664,222,300,1159",SAC_SIL_02347,CK_Symbols of Planets,21.11103/sphaera.101089,La sfera di M. Giovanni di Sacrobosco,2215,62,"Sacrobosco, Johannes de, Danti, Piervincenzo","Salviani, Baldo","Rastelli, Giovanni Bernardino",390,21.11103/sphaera.100250,Perugia,1574
20952,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=930,1135,609,589",SAC_SIL_01350,CK_Sphericity of the Earth,21.11103/sphaera.101089,La sfera di M. Giovanni di Sacrobosco,2215,17,"Sacrobosco, Johannes de, Danti, Piervincenzo","Salviani, Baldo","Rastelli, Giovanni Bernardino",390,21.11103/sphaera.100250,Perugia,1574


In [51]:
elements['imageURL'] = elements['imageURL'].apply(lambda x: 'https://devmeta.sphaera.mpiwg-berlin.mpg.de/resource/?uri=' + x)

In [52]:
elements['bookhandle'] = elements['doiv'].apply(lambda x: 'hdl.handle.net/' + x)

In [53]:
elements['parthandle'] = elements['pdoiv'].apply(lambda x: 'hdl.handle.net/' + x)

In [55]:
# download all images and save with identifier, add identifier to dataframe. 

filenames = []
for idx, entry in enumerate(elements.iterrows()):
    bb = "/" + entry[1].bb.split("=")[-1]
    link = '/'.join(entry[1].page_link.split('/')[:-4])
    ending = "/!300,300/0/default.jpg"
    link = link + bb + ending
    filenames.append(str(entry[1].imageURL.split('/')[-1])+".jpg")
    img_data = requests.get(link).content
    if not os.path.exists('elements_images_dataset/'):
        os.makedirs('elements_images_dataset/')
    with open('elements_images_dataset/'+ str(entry[1].imageURL.split('/')[-1])+".jpg", 'wb') as handler:
        handler.write(img_data)

elements['filenames']= filenames

In [57]:
elements

,imageURL,page_link,bb,cluster_name,cks,doiv,short_title,bid,pagenum,authors,publishers,printers,pid,pdoiv,place,year,bookhandle,parthandle,filenames
0,https://devmeta.sphaera.mpiwg-berlin.mpg.de/re...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=354,261,360,363",SAC_SIL_00197,CK_Equinoctial,21.11103/sphaera.101116,Iohannis de Sacro Busto libellus de sphaera,1898,43,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,932,21.11103/sphaera.101420,Wittenberg,1578,hdl.handle.net/21.11103/sphaera.101116,hdl.handle.net/21.11103/sphaera.101420,fac8639d-db7c-4ead-aa1e-863bb85ab76f.jpg
1,https://devmeta.sphaera.mpiwg-berlin.mpg.de/re...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=351,433,548,279",SAC_SIL_01942,"CK_Refraction, CK_Apparent Size of Stars, CK_S...",21.11103/sphaera.101116,Iohannis de Sacro Busto libellus de sphaera,1898,33,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,932,21.11103/sphaera.101420,Wittenberg,1578,hdl.handle.net/21.11103/sphaera.101116,hdl.handle.net/21.11103/sphaera.101420,89797d9e-8186-4d8b-9414-a4b7ab3ea77c.jpg
2,https://devmeta.sphaera.mpiwg-berlin.mpg.de/re...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=309,692,597,128",SAC_SIL_01616,"CK_Apparent Size of Stars, CK_Sphericity of th...",21.11103/sphaera.101116,Iohannis de Sacro Busto libellus de sphaera,1898,33,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,932,21.11103/sphaera.101420,Wittenberg,1578,hdl.handle.net/21.11103/sphaera.101116,hdl.handle.net/21.11103/sphaera.101420,8ba4a647-351e-4c25-b621-5fb968521201.jpg
3,https://devmeta.sphaera.mpiwg-berlin.mpg.de/re...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=473,212,336,243",SAC_SIL_01871,"CK_Apparent Size of Stars, CK_Sphericity of th...",21.11103/sphaera.101116,Iohannis de Sacro Busto libellus de sphaera,1898,33,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,932,21.11103/sphaera.101420,Wittenberg,1578,hdl.handle.net/21.11103/sphaera.101116,hdl.handle.net/21.11103/sphaera.101420,ccc6938a-32aa-4c96-8591-43251ca1d0f2.jpg
4,https://devmeta.sphaera.mpiwg-berlin.mpg.de/re...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=601,214,340,329",SAC_SIL_01952,CK_Relations Between the Spheres of the Sun an...,21.11103/sphaera.101116,Iohannis de Sacro Busto libellus de sphaera,1898,29,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,932,21.11103/sphaera.101420,Wittenberg,1578,hdl.handle.net/21.11103/sphaera.101116,hdl.handle.net/21.11103/sphaera.101420,c6e86f10-8a21-43e9-81df-1be233e7271f.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20949,https://devmeta.sphaera.mpiwg-berlin.mpg.de/re...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=616,1352,300,415",SAC_SIL_02926,"CK_Symbols of the Moon, CK_Symbols of Planets",21.11103/sphaera.101089,La sfera di M. Giovanni di Sacrobosco,2215,63,"Sacrobosco, Johannes de, Danti, Piervincenzo","Salviani, Baldo","Rastelli, Giovanni Bernardino",390,21.11103/sphaera.100250,Perugia,1574,hdl.handle.net/21.11103/sphaera.101089,hdl.handle.net/21.11103/sphaera.100250,d53d74f5-6307-41bb-8d1d-3fc58f572579.jpg
20950,https://devmeta.sphaera.mpiwg-berlin.mpg.de/re...,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...,"xywh=1065,613,594,594",SAC_SIL_01943,"CK_Circles of Equant Deferent Epicycle, CK_Lun...",21.11103/sphaera.101089,La sfera di M. Giovanni di Sacrobosco,2215,62,"Sacrobosco, Johannes de, Danti, Piervincenzo","Salviani, Baldo","Rastelli, Giovanni Bernardino",390,21.11103/sphaera.100250,Perugia,1574,hdl.handle.net/21.11103/sphaera.101089,hdl.handle.net/21.11103/sphaera.100250,7762eb42-bb15-4ce2-af7f-94d0de73ae8a.jpg
20951,https://devmeta.sphaera.mpiwg-berlin.mpg.de/re...,h

In [69]:
#elements.drop(['bb', 'page_link'], axis=1)

# order 

reordered = elements.iloc[:,[9,10,11,3,4,6,7,8,16,17,18]]
reordered = reordered.rename(columns={'cks':'CKs', 'bid':'book_identifier', 'pagenum':'page'})
reordered

,authors,publishers,printers,cluster_name,CKs,short_title,book_identifier,page,bookhandle,parthandle,filenames
0,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,SAC_SIL_00197,CK_Equinoctial,Iohannis de Sacro Busto libellus de sphaera,1898,43,hdl.handle.net/21.11103/sphaera.101116,hdl.handle.net/21.11103/sphaera.101420,fac8639d-db7c-4ead-aa1e-863bb85ab76f.jpg
1,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,SAC_SIL_01942,"CK_Refraction, CK_Apparent Size of Stars, CK_S...",Iohannis de Sacro Busto libellus de sphaera,1898,33,hdl.handle.net/21.11103/sphaera.101116,hdl.handle.net/21.11103/sphaera.101420,89797d9e-8186-4d8b-9414-a4b7ab3ea77c.jpg
2,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,SAC_SIL_01616,"CK_Apparent Size of Stars, CK_Sphericity of th...",Iohannis de Sacro Busto libellus de sphaera,1898,33,hdl.handle.net/21.11103/sphaera.101116,hdl.handle.net/21.11103/sphaera.101420,8ba4a647-351e-4c25-b621-5fb968521201.jpg
3,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,SAC_SIL_01871,"CK_Apparent Size of Stars, CK_Sphericity of th...",Iohannis de Sacro Busto libellus de sphaera,1898,33,hdl.handle.net/21.11103/sphaera.101116,hdl.handle.net/21.11103/sphaera.101420,ccc6938a-32aa-4c96-8591-43251ca1d0f2.jpg
4,"Melanchthon, Philipp, Sacrobosco, Johannes de",Company Heirs of Peter I. Seitz,Company Heirs of Peter I. Seitz,SAC_SIL_01952,CK_Relations Between the Spheres of the Sun an...,Iohannis de Sacro Busto libellus de sphaera,1898,29,hdl.handle.net/21.11103/sphaera.101116,hdl.handle.net/21.11103/sphaera.101420,c6e86f10-8a21-43e9-81df-1be233e7271f.jpg
...,...,...,...,...,...,...,...,...,...,...,...
20949,"Sacrobosco, Johannes de, Danti, Piervincenzo","Salviani, Baldo","Rastelli, Giovanni Bernardino",SAC_SIL_02926,"CK_Symbols of the Moon, CK_Symbols of Planets",La sfera di M. Giovanni di Sacrobosco,2215,63,hdl.handle.net/21.11103/sphaera.101089,hdl.handle.net/21.11103/sphaera.100250,d53d74f5-6307-41bb-8d1d-3fc58f572579.jpg
20950,"Sacrobosco, Johannes de, Danti, Piervincenzo","Salviani, Baldo","Rastelli, Giovanni Bernardino",SAC_SIL_01943,"CK_Circles of Equant Deferent Epicycle, CK_Lun...",La sfera di M. Giovanni di Sacrobosco,2215,62,hdl.handle.net/21.11103/sphaera.101089,hdl.handle.net/21.11103/sphaera.100250,7762eb42-bb15-4ce2-af7f-94d0de73ae8a.jpg
20951,"Sacrobosco, Johannes de, Danti, Piervincenzo","Salviani, Baldo","Rastelli, Giovanni Bernardino",SAC_SIL_02347,CK_Symbols of Planets,La sfera di M. Giovanni di Sacrobosco,2215,62,hdl.handle.net/21.11103/sphaera.101089,hdl.handle.net/21.11103/sphaera.100250,2e8f939a-1387-4f60-8f48-a540ae9dad08.jpg
20952,"Sacrobosco, Johannes de, Danti, Piervincenzo","Salviani, Baldo","Rastelli, Giovanni Bernardino",SAC_SIL_01350,CK_Sphericity of the Earth,La sfera di M. Giovanni di Sacrobosco,2215,17,hdl.handle.net/21.11103/sphaera.101089,hdl.handle.net/21.11103/sphaera.100250,d90ad014-0f79-46ff-b9a2-85a9bb51ca6e.jpg


In [71]:
reordered.to_csv('./elements.csv')